In [1]:
import sys

sys.path.append('..')

from src_refactored.configs import DataConfig, NetConfig, NetArchitecture, FilterConfig
from src_refactored.utils import train, get_wandb_logger
from src_refactored.configs import PACKAGE_PATH
from src_refactored.data.data_processor import DataProcessor

# DataLoader

In [3]:
filter=FilterConfig(
    n_bins= 30,
    n_gaps= 10,
    gap_size= 5, 
    rms_ratio= 0.,
    non_zero_ratio=0.8
)

data_config = DataConfig(
    path="/mnt/c/work/Fall_2021_csv",
    output_path=f"{PACKAGE_PATH}/resources/datasets",
    class_names=["cz_3", "falcon_9", "atlas_V",  "h2a", "globalstar"],
    regexes=[r'CZ-3B.*', r'FALCON_9.*', r'ATLAS_[5|V]_CENTAUR_R_B$',  r'H-2A.*', r'GLOBALSTAR.*'],
    validation_split=0.2,
    split_strategy="objectID",
    number_of_training_examples_per_class=10_000,
    filter_config=filter,
    convert_to_mag=False,
    fourier=False,
    std= False,
    rms= False,
    residuals= False,
    amplitude= False,
    lc= True,
    reconstructed_lc= False,
    push_to_max= False
)

In [4]:
dp = DataProcessor(data_config)
dp.load_data_MMT()
# dp.load_raw_data_MMT()
# dp.save_data_MMT()

In [6]:
train_set, val_set = dp.get_pytorch_datasets()

# Lightning module